In [ ]:
import os
import spacy
import torch
import logging
import pdfplumber
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from langdetect import detect
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

Modelo de Ollama cargado correctamente.


In [ ]:
# Directrorios de los documentos requeridos:
cvsPath = r'C:\Users\Estiven Angel\NLP_p2025\CVBestFit\cvs' # Ruta donde se almacenan los CVs sin procesar
#cvsTextPath = r'C:\Users\Estiven Angel\NLP_p2025\CVBestFit\cvsNormalizedText' # Ruta donde se almacenan los CVs procesados
trainingSetPath = r'C:\Users\Estiven Angel\NLP_p2025\CVBestFit\cvsTrainingSet\all' # Ruta del set de entrenamiento del modelo
jobDescriptionFile = r'C:\Users\Estiven Angel\NLP_p2025\CVBestFit\jobDescriptions\DP - Asesor Planeación SOP.pdf' # Directorio del documento descriptivo

#os.makedirs(cvsTextPath, exist_ok=True) # Crear directorio en caso de que no exista 

In [ ]:
# Fuciones de soporte:
def loadModels(useGpu=True):
    """ Carga el modelo de spaCy para español e inglés para GPU sí hay alguna disponible, caso contrario: carga modelos de CPU."""
    # Se verifica si Torch detecta la GPU:
    gpuAvailable = torch.cuda.is_available()

    if useGpu and gpuAvailable:
        try:
            spacy.require_gpu()  # Forzar SpaCy a usar GPU si está disponible
            modelEs = spacy.load("es_dep_news_trf")
            modelEn = spacy.load("en_core_web_trf")

            print(f"✅ GPU Disponible para spaCy: {torch.cuda.get_device_name(0)} 🖥️")

        except Exception as e:
            print("⚠️  Error al cargar modelos con GPU. Se cargaron modelos del CPU.")
            print("Detalles del error:", str(e))
            modelEs = spacy.load("es_core_news_md")
            modelEn = spacy.load("en_core_web_md")

    else:
        print("❌ No GPU activada o disponible para spaCy. Se utilizará CPU.")
        modelEs = spacy.load("es_core_news_md")
        modelEn = spacy.load("en_core_web_md")

    return modelEs, modelEn

def chooseModel(text):
    """ Detecta el idioma del texto y devuelve el modelo spaCy correspondiente ('es' o 'en')."""
    language = detect(text)

    if language == "es":
        model = modelEs # Seleccionar modelo en español
    elif language == "en":
        model = modelEn # Seleccionar modelo en ingles
    else:
        raise Exception(f"Idioma no soportado: {language}")
    
    return model

def extractText(docPath):
    """ Extrae y concatena el texto de todas las páginas de un archivo PDF."""
    with pdfplumber.open(docPath) as pdf:
        return "\n".join([page.extract_text() or '' for page in pdf.pages])
    
def normalizeText(text):
    """ Normaliza el texto eliminando stopwords y signos de puntuación, y devuelve los lemas en minúsculas """
    model = chooseModel(text) # Seleccionar el modelo en base al idioma
    doc = model(text) # Tokenización y POS Tagging

    # Convertir a minúsculas, remover stop-words y signos de puntuación
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(lemmas)

def concatAllTexts(textPath):
    """ Concatena el texto de todos los archivos .pdf en un directorio dado."""
    allTexts = []
    for file in os.listdir(textPath):
        if file.endswith('.pdf'): # Solo archivos PDF
            cPath = os.path.join(textPath, file)
            text = extractText(cPath) # Extraer el texto del documento actual
            allTexts.append(text)

        ######################## Suprimir avisos de pdfplumber sobre cropbox ############################
        logging.getLogger("pdfminer").setLevel(logging.ERROR)

    return allTexts

In [ ]:
# Se define la función de vectorización del corpus:
def tfIdfWeightedEmbedding(texts, modelW2V):
    """Vectoriza el corpus de textos usando TF-IDF y calcula los embeddings ponderados de cada documento."""
    # Se vectorizan con TF-IDF los textos (documentos)
    vectorizerTfIdf = TfidfVectorizer()
    tfIdfVectors = vectorizerTfIdf.fit_transform(texts)
    uniqueWords = vectorizerTfIdf.get_feature_names_out()

    # Se mapean los pesos TF-IDF:
    tfIdfWeights = []
    for doc in tfIdfVectors:
        wordWeight = {uniqueWords[i]: doc[0, i] for i in doc.nonzero()[1]}
        tfIdfWeights.append(wordWeight)

    # Se calculan los embeddings ponderados de cada documento:
    embedings = {}

    for i, doc in enumerate(texts):
        weights = [] # Pesos TF-IDF
        vectors = [] # Vectores ponderados de las palabras encontradas
        tokens = doc.split() # Palabras tokenizadas

        for word in tokens:
            if word in modelW2V.wv and word in tfIdfWeights[i]:
                vectors.append(modelW2V.wv[word] * tfIdfWeights[i][word]) # Se guarda el vector ponderado
                weights.append(tfIdfWeights[i][word]) # Se guarda el peso TF-IDF de la palabra

        if vectors: # Sí hay palabras con word embeddings y TF-IDF:
            embedings[i] = np.sum(vectors, axis=0) / sum(weights) # Calcular el promedio ponderado de los vectores
        else: # Sí no:
            embedings[i] = np.zeros(modelW2V.vector_size) # Regresar un vector de ceros para ese documento

    return embedings

In [ ]:
# Cargar el modelo de spacy para procesar el texto
modelEs, modelEn = loadModels() # Cargar los modelos para español e inglés

# Se entrena el modelo Word2Vec con todos los CVs y la descripción de la vacante:

# Almacenar los CVs:
trainingTexts = concatAllTexts(trainingSetPath) # Textos concatenados para entrenamiento
rawTexts = concatAllTexts(cvsPath) # Texto de los CVs de la vacante actual

# Almacenar también la descripción del trabajo para el entrenamiento:
jobDescription = extractText(jobDescriptionFile)
trainingTexts.append(jobDescription)
rawTexts.append(jobDescription)

# Se tokenizan los textos de trainingTexts para el entrenamiento del modelo:
tokenizedTrainingText = [text.lower().split() for text in trainingTexts] # Se convierten también a minúsculas

# Se entrena el modelo word2Vec:
modelW2V = Word2Vec(sentences=tokenizedTrainingText, vector_size=100, min_count=1, sg=1)

# No nOrmalizar...
# Seccionar en xml los cvs
# Comparar los candidatos entre sí.
# Revisar la opción de chunks
# Lexicons para el nombre

# Extraer embeddings ponderados de los CVs y descripciones de la vacante actual:
#normalizedTexts = [normalizeText(text) for text in rawTexts] # Normalizar el texto
tokenizedRawTexts = [text.lower() for text in rawTexts]
embeddings = tfIdfWeightedEmbedding(tokenizedRawTexts, modelW2V) # Extraer TF-IDF embeddings

In [ ]:
jobDescriptionVec = embeddings[len(embeddings)-1]  # El último vector ponderado corresponde a la descripción de la vacante

scores = {} # Se almacenan los scores (similitud coseno) de cada CV

for i, file in enumerate(os.listdir(cvsPath)):
    if file.endswith('.pdf'):
        cosSim = cosine_similarity([jobDescriptionVec], [embeddings[i]]).item() # Calcular similitud del CV contra la descripción
        nameCV = os.path.splitext(file)[0] # Nombre del CV

        scores[nameCV] = cosSim # Se guarda el score como valor y el nombre del CV como clave

    ######################## Suprimir avisos de pdfplumber sobre cropbox ############################
    logging.getLogger("pdfminer").setLevel(logging.ERROR)

# Se ordenan los scores de manera descendente:
scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))

In [ ]:
# Graficar los resultados de los scores:
scoresDF = pd.DataFrame(list(scores.items()), columns=['CV', 'Puntuación']) # Crear un DataFrame
display(scoresDF)

fig, ax = plt.subplots(figsize=(12, 8))
sns.barplot(x='CV', y='Puntuación', data=scoresDF, hue='CV', palette='Spectral',ax=ax)
plt.title("Índice de afinidad Candidato-Vacante")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.xticks(rotation=90)
plt.tight_layout()

plt.show()

In [ ]:
# Crear heurística para puntuar cada skill con la que cuente el candidato:
def getSkillVector(skill, modelW2V):
    "Obtiene el vector promedio de todas las palabras de la skill"
    skillVector = np.mean([modelW2V.wv[word] for word in skill if word in modelW2V.wv.key_to_index], axis=0)
    return skillVector

def getSkillSynonyms(allSkills, corpus, modelW2V, threshold=0.7):
    """ Devuelve un diccionario de los sinonimos de la skill (palabra o frase) mediante similitud vectorial"""
    synonyms = {}
    flatCorpus = [word for sentence in corpus for word in sentence] # Todas las palabras del corpus en una sola lista

    for skill in allSkills:
        similarSkills = []
        skillVector = getSkillVector(skill, modelW2V)

        for word in flatCorpus:
            if word in modelW2V.wv.key_to_index:
                try:
                    cosSim = modelW2V.wv.cosine_similarities(skillVector, [modelW2V.wv[word]])[0]
                    if cosSim > threshold:
                        similarSkills.append(word) # Guardar el sinónimo sí excede el threshold

                except KeyError:
                    continue
            
        synonyms[skill] = [skill] # Agregar la propia skill al diccionario
        synonyms[skill].extend(similarSkills) # Agregar las skills similares al diciconario

    return synonyms

def evaluateSkills(skills, weight, cvSkills, synonyms):
        score = 0
        total = 0

        for skill in skills:
            total += weight
            found = any(syn in cvSkills for syn in synonyms[skill])
            score += weight if found else -weight
        
        return score, total

def skillsScore(cvSkills, hardSkillsMust, hardSkillsNice, softSkillsMust, softSkillsNice):
    """ Evalúa con una heurística la puntuación por cada skill con la que cuente el CV y devuelve dicha puntuación"""

    # Enriquecer las skills con sinónimos:
    allSkills = cvSkills + hardSkillsMust + hardSkillsNice + softSkillsMust + softSkillsNice
    synonyms = getSkillSynonyms(allSkills, tokenizedTrainingText, modelW2V)
    
    # synonymsDF = pd.DataFrame(list(synonyms.items()), columns=['Palabra', 'Sinónimos']) # Crear un DataFrame
    # display(synonymsDF)  

    # Definir los pesos de las skills:
    weights = {
        "hardMust": 5,
        "hardNice": 3,
        "softMust": 2,
        "softNice": 1
    }

    # Evaluar las categorías de skills:
    categories = [
        (hardSkillsMust, weights["hardMust"]),
        (hardSkillsNice, weights["hardNice"]),
        (softSkillsMust, weights["softMust"]),
        (softSkillsNice, weights["softNice"]),
    ]

    score = 0
    totalPossible = 0

    for skillList, weight in categories:
        if skillList:
            subScore, subTotal = evaluateSkills(skillList, weight, cvSkills, synonyms)
            score += subScore
            totalPossible += subTotal
        else:
            continue

    # print(f"Total posible: {totalPossible}")
    # print(f"Score bruto: {score}")

    skillsScore = max(score, 0) / totalPossible if totalPossible else 0

    return skillsScore

In [ ]:
cvSkills = ["R, SQL, Python"]
hardSkillsMust = ["Calibracion de sensores, electroneumatica, Conocimiento en PLCs como Siemens o Allen Bradley"]
hardSkillsNice = ["Pandas, Scikit-learn, Tensorflow, Pytorch, and OpenCV"]
softSkillsMust = ["Big Data, Feature Engineering, optimización, machine learning"]
softSkillsNice = ["Apache Spark, Amazon Web Services, Google Cloud"]

skillScore = skillsScore(cvSkills, hardSkillsMust, hardSkillsNice, softSkillsMust, softSkillsNice)
print(skillScore)